In [ ]:
import lephare as lp
import numpy as np
import matplotlib
from matplotlib import pylab as plt

%matplotlib inline

In [ ]:
seds = lp.readBC03(
    "../../../lephare-data/sed/GAL/BC03_CHAB/bc2003_lr_m42_chab_tau01_dust00.ised_ASCII", 0, [-999, -999]
)

In [ ]:
[plt.loglog(*sed.data()) for sed in seds];

In [ ]:
sed = seds[20]
# print(sed.age)
x, y = sed.data()
sed.calc_ph()
cont = sed.add_neb_cont()
x2, y2 = sed.data()
plt.semilogy(x, y)
plt.semilogy(x2, y2)
plt.semilogy(x2, cont, ".")
plt.xlim(0, 10000)
plt.ylim(1.0e-12, 1.0e-8)

In [ ]:
np.where(y2 != y)

In [ ]:
sed = lp.GalSED("", 0)
sed.read("../../tests/data/sed/GAL/COSMOS_SED/S0_A_0.sed")
sed.calc_ph()
# sedx, sedy = sed.data()
# cont = sed.add_neb_cont()
# sed2x, sed2y = sed.data()
# plt.semilogx(sedx,sedy)
# plt.plot(sed2x,sed2y)

In [ ]:
sed.lamb_flux[0].lamb

In [ ]:
cont2 = sed.add_neb_cont2()

In [ ]:
x = [c.lamb for c in cont]
y = [c.val for c in cont]
x2 = [c.lamb for c in cont2]
y2 = [c.val for c in cont2]

In [ ]:
plt.semilogx(x, y, ".-")
plt.semilogx(x2, y2, ".-")

In [ ]:
plt.plot(lp.ga_lamb, lp.ga_H_val)
plt.plot(lp.ga_lamb, lp.ga_2q_val)
plt.plot(lp.ga_lamb, 0.1 * np.array(lp.ga_HeI_val))
plt.plot(lp.ga_lamb, np.array(lp.ga_H_val) + np.array(lp.ga_2q_val) + 0.1 * np.array(lp.ga_HeI_val))

In [ ]:
sedx, sedy = sed.data()

In [ ]:
lp.read